In [ ]:
# import libraries
import requests
import os
import hashlib
import pathlib
import json
from datetime import datetime
import pytz
from urllib.parse import urlparse
import time
import pandas as pd
import googlemaps
import concurrent.futures

In [ ]:
# api key
key = 'Jessica_Key'

In [ ]:
from functions.routes_api_functions import Location
from functions.routes_api_functions import calculateRoute
from functions.routes_api_functions import parseDataframeToLocations
from functions.routes_api_functions import createDataFrame

In [ ]:
from functions.routes_api_functions import cache_file_path
from functions.routes_api_functions import save_to_cache
from functions.routes_api_functions import load_from_cache
from functions.routes_api_functions import fetch_all_routes

In [ ]:
from functions.distance_matrix_functions import cache_distance_info
from functions.distance_matrix_functions import batch_distance_requests

In [ ]:
## import O-D
# import origin and destination data
origins = pd.read_csv('../Data/origins_gdf.csv')
originsLoc = parseDataframeToLocations(origins)
destinations = pd.read_csv('../Data/destinations_gdf.csv')
destinationsLoc = parseDataframeToLocations(destinations)

## 1. Check multiple request availability

### Routes API

In [ ]:
def location_to_dict(location):
    return {
        "latLng": {
            "latitude": location.lat,
            "longitude": location.lon
        }
    }

origin_dicts = [location_to_dict(loc) for loc in originsLoc]
destination_dicts = [location_to_dict(loc) for loc in destinationsLoc]

# create a list of O-D pairs to pass it as a whole to the routes api
od_pairs = []
for o, d in zip(origin_dicts, destination_dicts):
    od_pairs.append({
        "origin": {"location": o},
        "destination": {"location": d},
        "travelMode": "DRIVE",
        "routingPreference": "TRAFFIC_AWARE",
        "departureTime": "2023-11-30T15:00:00-08:00",
        "computeAlternativeRoutes": False,
        "routeModifiers": {
            "avoidTolls": False,
            "avoidHighways": False,
            "avoidFerries": False
        },
        "languageCode": "en-US",
        "units": "IMPERIAL"
    })

In [ ]:
# headers
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': key,
    'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline'
}
url = "https://routes.googleapis.com/directions/v2:computeRoutes"

results = []

# combine all O-D pairs into a single request data structure
request_data = od_pairs
 
# make the request
response = requests.post(url, headers=headers, data=json.dumps(request_data))
if response.status_code == 200:
        response_json = response.json()
        results.append(response_json)
else:
    print("Error:", response.status_code, response.text)
    results.append(None)

In [ ]:
results

In [ ]:
# check if it works when directly inputting o-d
# headers
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': key,
    'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline'
}
url = "https://routes.googleapis.com/directions/v2:computeRoutes"

results = []

request_data = {
    "origin": {
            "location": {
                "latLng": {
                    "latitude": originsLoc[0].lat,
                    "longitude": originsLoc[0].lon
                }
                "latLng": {
                    "latitude": originsLoc[0].lat,
                    "longitude": originsLoc[0].lon
                }
            }
        },
        "destination": {
            "location": {
                "latLng": {
                    "latitude": destinationsLoc[0].lat,
                    "longitude": destinationsLoc[0].lon
                }
                "latLng": {
                    "latitude": destinationsLoc[0].lat,
                    "longitude": destinationsLoc[0].lon
                }
            }
        },
        "travelMode": "DRIVE",
        "routingPreference": "TRAFFIC_AWARE",
        # departure time: UTC-8
        # 2023-11-30 15:00
        "departureTime": "2023-11-30T15:00:00-08:00",
        "computeAlternativeRoutes": False,
        "routeModifiers": {
            "avoidTolls": False,
            "avoidHighways": False,
            "avoidFerries": False
        },
        "languageCode": "en-US",
        "units": "IMPERIAL"
    }

response = requests.post(url, headers=headers, data=json.dumps(request_data))
if response.status_code == 200:
        response_json = response.json()
        results.append(response_json)
else:
    print("Error:", response.status_code, response.text)
    results.append(None)

Seems Routes API does not support multiple requests.

### Distance Matrix

In [ ]:
# distance matrix
gmaps = googlemaps.Client(key=key)

In [ ]:
def location_to_string(location):
    return f"{location.lat},{location.lon}"

# convert Location objects to strings
## will add this to distance_matrix_functions.py if it is needed for reproduction
test_o = [location_to_string(loc) for loc in originsLoc]
test_d = [location_to_string(loc) for loc in destinationsLoc]

In [ ]:
test2 = gmaps.distance_matrix(test_o, test_d, mode="driving")
test2

Distance Matrix is able to pass multiple requests per each request.

## 2. Batch Processing - Distance Matrix

In [ ]:
results = batch_distance_requests(origins=test_o, destinations=test_d, mode="driving", batch_size=2, departure_time=(2023, 12, 1, 14, 0, 0))

In [ ]:
results

## 3. Routes API

In [ ]:
## come up with other ways to run multiple requests at once

In [ ]:
results = fetch_all_routes(apikey=key, origins=originsLoc, destinations=destinationsLoc, traffic='TRAFFIC_AWARE', departure_time=(2023, 12, 1, 13, 0, 0))

In [ ]:
results